In [2]:
import os
import pyspark
import boto3
import s3fs
import pandas
from subprocess import run as run_sh
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from os import listdir
from os.path import join as path_join

ModuleNotFoundError: No module named 's3fs'

In [ ]:
%%sh
echo '#!/bin/sh' > credentials.sh
saml2aws script >> credentials.sh
. ./credentials.sh
env | grep AWS

In [ ]:
%%sh

vi credentials.sh

In [ ]:
gen_tmp_sec_creds = run_sh(["saml2aws", "script"],capture_output=True)
if gen_tmp_sec_creds.stderr != b'':
   raise Exception("Found error when obtaining AWS " +
         "credentials, they most be in the config.yaml " +
         "file or saml2aws must be running." +
         f"\nSAML2AWS error: {gen_tmp_sec_creds.stderr}")
access_key = os.environ["AWS_ACCESS_KEY_ID"]
secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
token = os.environ["AWS_SESSION_TOKEN"]

In [ ]:
os.environ.get

In [ ]:
print(access_key)
print(secret_key)

In [1]:
driver_jars_path = path_join("..","..","Development","jar")
print(driver_jars_path)
all_jars = listdir(driver_jars_path)
print(all_jars)
all_pathed_jars = [path_join(driver_jars_path, jar)
                           for jar in all_jars]
jars_paths = ",".join(all_pathed_jars)

NameError: name 'path_join' is not defined

In [ ]:
spark = SparkSession.builder.appName("Sensor Data Validation").config('spark.jars', jars_paths) \
                .config('spark.jars.packages',
                        'org.apache.hadoop:hadoop-aws:3.3.6') \
                .config(
                    'spark.hadoop.fs.s3a.aws.credentials.provider',
                    'org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider'
                    ) \
                .config('spark.hadoop.fs.s3a.access.key', access_key) \
                .config('spark.hadoop.fs.s3a.secret.key', secret_key) \
                .config('spark.hadoop.fs.s3a.session.token', token).getOrCreate()

In [ ]:
dataXom = spark.read.option("header",True).csv("s3a://322589048303-aa-xom-data-extracts-profiling/XTO_ENERGY_INC_MIDLAND_08_02_2023_21_48_43_247_Canada_Central/")

In [ ]:
dataXom.printSchema

In [ ]:
dataXom.count()

In [ ]:
data = spark.read.parquet("s3a://322589048303-us-east-1-leucipa-emr/PILOT_HUDI_v2/WELL_DEVICE_ESP/")

In [ ]:
##data = spark.read.parquet("s3a://322589048303-us-east-1-leucipa-emr/PILOT_HUDI/WELL_DEVICE_ESP/")

In [ ]:
data.printSchema()

In [ ]:
data.count()

In [ ]:
data.createOrReplaceTempView("sensor_data")
df = spark.sql("select tag_name_original, tag_name_og, tag_unit_og, tag_unit_original, count(1) from sensor_data group by tag_name_original, tag_name_og, tag_unit_og, tag_unit_original")
df.show(truncate=False)

In [ ]:
df.write.csv("tag_names_units.counting")

In [ ]:
data.

In [ ]:
dfone = data.withColumn("date",to_date("record_dt_utc")).withColumn("time",date_format("record_dt_utc","HH:mm:ss")).where("_hoodie_file_name=='16f2b327-7fe5-442e-ab4f-6d7363b00df5-0_47-18-2363_20230712074229438.parquet'") 
df1 = dfone.groupBy("apinumber","date","time").pivot("tag_name_original").agg({"tag_value_original":"first"})

In [ ]:
df1.filter("apinumber==3002546655 and date=='2022-11-01' and time=='19:03:46'").show()

In [ ]:
dfone.printSchema()

In [ ]:
dfone.createOrReplaceTempView("temporal")

In [ ]:
dfonediff = spark.sql("select apinumber,date,time,count(distinct tag_name_original) as q from temporal group by apinumber,date,time having count(distinct tag_name_original)>2")

In [ ]:
dfonediff.show(truncate=False)

In [ ]:
df2 = data.groupBy("_hoodie_file_name").count()
df2.show(truncate=False)

In [ ]:
df2 = data.groupBy("apinumber").count()
df2.show(truncate=False)

In [ ]:
df1.printSchema()

In [ ]:
dfResult = df1.na.fill(-9999)

In [ ]:
dfResult.count()

In [ ]:
dfResult.head()

In [ ]:
dfone.count()

In [ ]:
dfResult.show(truncate=False)

In [ ]:
dfResult.where("apinumber==3002546655").show(truncate=False)

In [ ]:
dfResult.groupBy("apinumber","record_dt_utc").count().show(truncate=False)

In [ ]:
dfCatalog = spark.read.parquet("s3a://322589048303-pre-prod-xom-esp-data/refined/well_catalog/")

In [ ]:
dfCatalog.count()

In [ ]:
dfCatalog.printSchema()

In [ ]:
dfCatalog.show()

In [ ]:
dfCat = dfCatalog.withColumn("TimeStamp",current_timestamp())

In [ ]:
dfCat.printSchema()

In [ ]:
dfCat.show()

In [ ]:
dfCat.count()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [ ]:
WindowSpec = Window.partitionBy("TimeStamp").orderBy("DeviceID")
dfWell = dfCat.withColumn("WellInternalID",row_number().over(WindowSpec)) \
.withColumn("WellScanEnabled", expr("cast(1 as integer)")) \
.withColumn("WellMonitorEnabled", expr("cast(1 as integer)")) \
.withColumn("WellSCADA_Freq_Ms" , expr("cast(1200000 as integer)")) \
.withColumn("WellDeadTimeFactor" , expr("cast(3 as integer)")) \
.withColumn("WellAlarmEmailInterval_Min" , expr("cast(20 as integer)")) \
.withColumn("WellBackScanMin" , expr("cast (60 as integer)")) \
.withColumn("WellAutoMatchEnabled" , expr("cast(1 as integer)")) \
.withColumn("WellScadaDataSource" , expr("cast(2 as integer)")) \
.withColumn("WellSAPWellNumber" , expr("'Leucipa Hierarchy'"))  \
.withColumn("WellAPIWellNumber" , expr("'Leucipa Hierarchy'")) \
.withColumn("WellMAPSRefNumber", expr("'Leucipa Hierarchy'")) \
.withColumn("WellNotAdjustHLBasedOAPCModel", expr("cast(null as integer)")) \
.withColumn("Well247Enabled", expr("cast(null as integer)")) \
.withColumn("WellInactive", expr("cast(null as integer)")) \
.withColumn("WellAutoTask", expr("cast(1 as integer)")) \
.withColumn("WellMultiWellOpt", expr("cast(1 as integer)")) \
.withColumn("WellUID" , expr("'' as WellUID"))

In [ ]:
dfWell.printSchema()

In [ ]:
dfWellSQL = spark.read.format("jdbc") \
.option("url", "jdbc:sqlserver://bhicladev01.ent.bhicorp.com:50900;databaseName=EspExpert;encrypt=true;trustServerCertificate=true") \
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
.option("dbtable", "tblWellCopy") \
.option("user", "EspExpert") \
.option("password", "espexpert321") \
.load()

In [ ]:
dfWellSqlMaxID = dfWellSQL.agg(max("WellInternalWellID"))
MaxID = dfWellSqlMaxID.rdd.map(lambda x: x).collect()[0][0]

In [ ]:
dfWellStep2 = dfWell.withColumn("WellInternalWellID",expr(f"WellInternalID + {MaxID}"))

In [ ]:
dfWellCopy = dfWellStep2.select(col("WellID").alias("WellExternalWellID") \
              ,"WellName" \
              ,col("FieldName").alias("WellFieldName") \
              ,col("CustomerName").alias("WellCustomerName") \
              ,"WellSCADA_Freq_Ms" \
              ,"WellScanEnabled" \
              ,"WellMonitorEnabled" \
              ,"WellDeadTimeFactor" \
              ,"WellAlarmEmailInterval_Min" \
              ,col("FieldID").alias("WellFieldID") \
              ,col("CustomerID").alias("WellCustomerID") \
              ,"WellBackScanMIN" \
              ,"WellAutoMatchEnabled" \
              ,"WellNotAdjustHLBasedOAPCModel" \
              ,"WellScadaDataSource" \
              ,"WellSAPWellNumber" \
              ,"WellAPIWellNumber" \
              ,"WellMAPSRefNumber" \
              ,"Well247Enabled" \
              ,"WellInactive" \
              ,"WellAutoTask" \
              ,"WellMultiWellOpt" \
              ,"WellUID" \
              ,col("DeviceID").alias("WellDeviceID") \
              ,col("DeviceDisplayName").alias("WellDeviceDisplayName"))

In [ ]:
dfWellCopy.printSchema()

In [ ]:
dfWellCopy.show()

In [ ]:
dfWellCopy.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:sqlserver://bhicladev01.ent.bhicorp.com:50900;databaseName=EspExpert;encrypt=true;trustServerCertificate=true") \
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
.option("dbtable", "tblWellCopy") \
.option("user", "EspExpert") \
.option("password", "espexpert321") \
.save()

In [ ]:
dfWellSQL.count()

In [ ]:
dfWellSQL.printSchema()

In [ ]:
dfSensor = data.alias("a").join(dfWellSQL.alias("b"), col("a.pump_id") == col("b.WellDeviceID"), "left").select("a.*",col("WellInternalWellID"))

In [ ]:
dfSensor.printSchema()

In [ ]:
dfSensor.count()

In [ ]:
dfone = dfSensor.withColumn("date",to_date("record_dt_utc")).withColumn("time",date_format("record_dt_utc","HH:mm:ss"))##.where("_hoodie_file_name=='16f2b327-7fe5-442e-ab4f-6d7363b00df5-0_47-18-2363_20230712074229438.parquet'") 
dfPSensor = dfone.groupBy("WellInternalWellID","date","time").pivot("tag_name_original").agg({"tag_value_original":"first"})

In [ ]:
dfPSensor.show(truncate=False)

In [ ]:
dfSensor.groupBy("pump_id","WellInternalWellID").count().show(truncate=False)